In [ ]:
with open("../StackExchange/final_data/selected_id.txt", 'r') as f:
    all_ids = f.readlines()

data_selectedId = sorted([int(tid.replace("\n","")) for tid in all_ids])
print(len(data_selectedId))

In [ ]:
df = pd.read_csv("../StackExchange/final_data/stackexchange.csv")

In [ ]:
df_tag_dict = pd.read_csv("../StackExchange/final_data/tag_dict.csv")
tag_list = list(df_tag_dict["Tag"].values)
print(len(tag_list))

In [ ]:
eval_selectedId = {i for i in range(len(df))}
print(len(eval_selectedId))
eval_selectedId = eval_selectedId.difference(data_selectedId)
print(len(eval_selectedId))

In [ ]:
df_select = df.iloc[[eid for eid in eval_selectedId]]

In [ ]:
df_tag = pd.read_csv("../StackExchange/final_data/tag_dict.csv")
print(len(df_tag))
df_tag["Corrected"] = df_tag['Corrected'].fillna(-1)
df_tag = df_tag[df_tag["Corrected"] != -1]
tag_corrected = {k:v for k,v in zip(df_tag["Tag"],df_tag["Corrected"])}

In [ ]:
data_per_tag_limit = 6
tag_dict = {}
eval_set = []
eval_true_tag = []

for body,title,tag_string in zip(df_select["Body"],df_select["Title"],df_select["Tags"]):
    input_string = title+" "+body
    
    tag_string = tag_string.replace("<","")
    tag_string = tag_string.replace(">"," ")
    overlimit = True
    skip = False
    tag_output = ""
    
    for tag in tag_string.strip().split(" "):
        if tag in tag_list:
            if tag in tag_corrected.keys() and tag_corrected[tag] != "no":
                tag = tag_corrected[tag]
            
            tag_output += tag.strip() + " "
            tag_dict[tag] = tag_dict.setdefault(tag, 0) + 1
                
            if tag_dict[tag] < data_per_tag_limit:
                overlimit = False
        else:
            skip = True
            break
            
    if skip:
        continue
    
    if overlimit:
        continue
    else:
        eval_set.append(input_string.strip())
        eval_true_tag.append(tag_output.strip())

In [ ]:
print(len(eval_set),len(eval_true_tag))

In [ ]:
eval_set[:5]

In [ ]:
eval_true_tag[:5]

In [ ]:
df_test = pd.DataFrame()
df_test["Input"] = eval_set
df_test["Tag_True"] = eval_true_tag

df_test.to_csv("../StackExchange/final_data/testing/eval_"+str(len(eval_set))+".csv",index = None)
df_test.head()